In [155]:
import scipy.io.idl as idl
import numpy as np
from scipy.interpolate import griddata
import idl_plane_to_structured_vtk as idl_to_vtk

reload(idl_to_vtk)

<module 'idl_plane_to_structured_vtk' from 'idl_plane_to_structured_vtk.py'>

# Try to store a single time from Jason's IDL scripts
Store $\vec{B}$, $T$, and $n$.

In [156]:
b_x_measurement_points_timestamp = '115110949362'
b_y_measurement_points_timestamp = '115110949522'
b_z_measurement_points_timestamp = '115110938869'
triple_measurement_points_timestamp = '115112535917'
b_x_measurements_timestamp = '115112539367'
b_y_measurements_timestamp = '115112539384'
b_z_measurements_timestamp = '115112539331'
n_measurements_timestamp = '115112539443'
t_measurements_timestamp = '115112539427'

In [40]:
data_path = '../../cataloging_physics_quantities_in_2D/output/intermediate/'
data_ending = '.sav'
b_x_measurement_points_file = data_path + b_x_measurement_points_timestamp + data_ending
b_y_measurement_points_file = data_path + b_y_measurement_points_timestamp + data_ending
b_z_measurement_points_file = data_path + b_z_measurement_points_timestamp + data_ending
triple_measurement_points_file = data_path + triple_measurement_points_timestamp + data_ending 
b_x_measurements_file = data_path + b_x_measurements_timestamp + data_ending
b_y_measurements_file = data_path + b_y_measurements_timestamp + data_ending
b_z_measurements_file = data_path + b_z_measurements_timestamp + data_ending
n_measurements_file = data_path + n_measurements_timestamp + data_ending
t_measurements_file = data_path + t_measurements_timestamp + data_ending

In [41]:
b_x_measurement_points = idl.readsav(b_x_measurement_points_file)
b_y_measurement_points = idl.readsav(b_y_measurement_points_file)
b_z_measurement_points = idl.readsav(b_z_measurement_points_file)
triple_measurement_points =  idl.readsav(triple_measurement_points_file)
b_x_measurements = idl.readsav(b_x_measurements_file)
b_y_measurements = idl.readsav(b_y_measurements_file)
b_z_measurements = idl.readsav(b_z_measurements_file)
n_measurements = idl.readsav(n_measurements_file)
t_measurements = idl.readsav(t_measurements_file)

In [42]:
for time_point in xrange(21):
    file_name = '../output/2016-01-07/temperature_%i.vts' % time_point
    (x_min, x_max,
     y_min, y_max) = idl_to_vtk.determine_sample_bounds([t_measurements])
    (t_resampled, 
     x_grid, y_grid) = idl_to_vtk.resample_scalar(t_measurements,
                                                  time_point,
                                                  x_min, x_max, y_min, y_max)
    (t_resampled, 
     x_grid, y_grid) = idl_to_vtk.remove_nans(t_resampled, x_grid, y_grid)
    mesh = idl_to_vtk.prepare_mesh(x_grid, y_grid, 0.25)
    t_reshaped = idl_to_vtk.reshape_scalar(t_resampled)
    idl_to_vtk.write_to_structured_grid(file_name, t_reshaped, 't', mesh)

In [43]:
for time_point in xrange(21):
    file_name = '../output/2016-01-07/magnetic_field_offset_%i.vts' % time_point
    vector_dicts = [b_x_measurements, b_y_measurements, b_z_measurements]
    (x_min, x_max,
     y_min, y_max) = idl_to_vtk.determine_sample_bounds(vector_dicts)
    (b_resampled, 
     x_grid, y_grid) = idl_to_vtk.resample_vector(vector_dicts[0], 
                                                  vector_dicts[1],
                                                  vector_dicts[2],
                                                  time_point,
                                                  x_min, x_max, y_min, y_max) 
    #(b_resampled, 
    # x_grid, y_grid) = idl_to_vtk.remove_nans_vector(b_resampled, x_grid, y_grid)
    b_resampled[0] = b_resampled[0][8:92,8:92]
    b_resampled[1] = b_resampled[1][8:92,8:92]
    b_resampled[2] = b_resampled[2][8:92,8:92]
    x_grid = x_grid[8:92,8:92]
    y_grid = y_grid[8:92,8:92]
    mesh = idl_to_vtk.prepare_mesh(x_grid, y_grid, 0.3)
    b_vector = idl_to_vtk.reshape_vector(b_resampled[0], b_resampled[1], b_resampled[2])
    idl_to_vtk.write_to_structured_grid(file_name, b_vector, 'B', mesh)

RuntimeError: output file '../output/2016-01-07/magnetic_field_offset_0.vts' already exists

# Test command line call

In [160]:
print b_x_measurements_file
print b_y_measurements_file
print b_z_measurements_file

../../cataloging_physics_quantities_in_2D/output/intermediate/115112539367.sav
../../cataloging_physics_quantities_in_2D/output/intermediate/115112539384.sav
../../cataloging_physics_quantities_in_2D/output/intermediate/115112539331.sav


In [161]:
print t_measurements_file

../../cataloging_physics_quantities_in_2D/output/intermediate/115112539427.sav


In [152]:
!ipython idl_plane_to_structured_vtk.py vector ../../cataloging_physics_quantities_in_2D/output/intermediate/115112539367.sav ../../cataloging_physics_quantities_in_2D/output/intermediate/115112539384.sav ../../cataloging_physics_quantities_in_2D/output/intermediate/115112539331.sav 0.25 21 B_25 B 3 3 None 25 25

In [163]:
!ipython idl_plane_to_structured_vtk.py scalar ../../cataloging_physics_quantities_in_2D/output/intermediate/115112539427.sav 0.25 21 t_25 t 3 3 None 25 25

In [164]:
!ls

B_25_000000.vts                                B_25_000010.vts                                B_25_000020.vts                                t_25_000004.vts                                t_25_000014.vts
B_25_000001.vts                                B_25_000011.vts                                example_vtk_structutred_file_generation.ipynb  t_25_000005.vts                                t_25_000015.vts
B_25_000002.vts                                B_25_000012.vts                                example_vtk_unstructured_file_generation.ipynb t_25_000006.vts                                t_25_000016.vts
B_25_000003.vts                                B_25_000013.vts                                idl_plane_to_structured_vtk.py                 t_25_000007.vts                                t_25_000017.vts

B_25_000005.vts                                B_25_000015.vts                                idl_sav_to_vtk_unstructured.py                 t_25_000009.vts                           